In [1]:
#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
//using BoSSS.XNSE_Solver;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using MomentFittingVariants = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants;
Init();

In [2]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "VanishingSphere2D" : buildname; //Popcorn2D  VanishingSphere2D
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "convergenceAggSteadyStokesExactBC" : dbname; //Collison´2
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
using BoSSS.Solution.Gnuplot;


In [4]:
//ExecutionQueues
var myBatch = ExecutionQueues[5];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

{ Session Count = 90; Grid Count = 10; Path = \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC }

In [5]:
myDB.Sessions.Where(s => s.SuccessfulTermination == true)

#0: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.2_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:03:04 PM	4e6c9e76...
#1: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.5_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:04:14 PM	136662d2...
#2: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.4_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:03:59 PM	2e86a22b...
#3: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.3_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:03:43 PM	99b0760f...
#4: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg0.5_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:02:52 PM	fa95be63...
#5: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg

In [6]:
int timeStep = -1;
// BoSSSshell.WorkflowMgm.hConvergence.Update(timeStep);

In [7]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
Tab.Columns.Summary()

Count: 231
IsReadOnly: False
IsSynchronized: False
SyncRoot: System.Data.DataColumnCollection


In [8]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns( "AgglomerationThreshold", "DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2err_"+ "VelocityX", "L2Err_" +"Pressure"); // 

Tab2.Print();  
Tab2.SaveToFile(table_name + $"_at{timeStep}_summary.json");

    AgglomerationThreshold DGdegree:Velocity* Grid:hMin            Grid:NoOfCells L2err_VelocityX        L2err_Pressure         
0:  0.2                    3                  0.008333333333333304 57600          3.697938199644299E-09  2.5588808254847334E-07 
1:  0.5                    3                  0.008333333333333304 57600          5.216859594367646E-09  2.5425053894028194E-07 
2:  0.4                    3                  0.008333333333333304 57600          4.5482850658530185E-09 2.506460091890163E-07  
3:  0.3                    3                  0.008333333333333304 57600          4.0801030801596335E-09 2.4923627615002925E-07 
4:  0.5                    2                  0.008333333333333304 57600          1.2007818915739127E-06 1.3790259368558115E-05 
5:  0.3                    2                  0.008333333333333304 57600          1.1182617957953E-06    1.1875199557467882E-05 
6:  0.4                    2                  0.008333333333333304 57600          1.1514820437518

In [9]:
using BoSSS.Solution.Gnuplot;
using System.Linq;

// arrange line color and types
var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle };


In [10]:
string xVarName = "Grid:hMin";
string yVarName = "L2err_VelocityX";
int dgDegree = 1;

// Filter rows depending on selection
var Tab3 = Tab2.ExtractRows((iRow,RowEntries)=> 
                //Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0 &&
                Convert.ToDouble(RowEntries["DGdegree:Velocity*"]) == dgDegree
            );

// Sort in ascending order of AgglomerationThreshold
System.Data.DataView view = Tab3.DefaultView;
view.Sort = "AgglomerationThreshold ASC"; 
Tab3 = view.ToTable();

Tab3.Print()


    AgglomerationThreshold DGdegree:Velocity* Grid:hMin            Grid:NoOfCells L2err_VelocityX       L2err_Pressure        
0:  0                      1                  0.06666666666666665  900            0.0891179483606521    0.03273951821864419   
1:  0                      1                  0.008333333333333304 57600          0.003980975882129364  0.0017792030322730213 
2:  0                      1                  0.1333333333333333   225            0.1490257615429513    0.09714230438277147   
3:  0                      1                  0.016666666666666607 14400          0.013552980362219486  0.004622049816939742  
4:  0                      1                  0.033333333333333215 3600           0.03965000328280205   0.012414089444504584  
5:  0.1                    1                  0.008333333333333304 57600          0.0039681910710860635 0.0014894358476796434 
6:  0.1                    1                  0.016666666666666607 14400          0.013447581008608402  0.00386

In [11]:
Plot2Ddata plot = new Plot2Ddata();

In [12]:
var plot = Tab3.ToPlot(xVarName, yVarName,  "AgglomerationThreshold");
plot.LogY = true;
plot.LogX = true;
        for (int i = 0; i < plot.dataGroups.Length; i++){
            var dataGroup = plot.dataGroups[i];
            dataGroup.Format.LineColor = allColors[i % allColors.Length];
            dataGroup.Format.PointType = myPointTypes[i % myPointTypes.Length];
        }   
plot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0 
 
 
 AgglomerationThreshold0 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.1 
 
 
 AgglomerationThreshold0.1 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.2 
 
 
 AgglomerationThreshold0.2 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.3 
 
 
 AgglomerationThreshold0.3 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.4 
 
 
 AgglomerationThreshold0.4 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.5 
 
 
 AgglomerationThreshold0.5

In [13]:
plot.SaveToGIF(yVarName + "_k" + dgDegree + "_2D.png");
plot.SavePgfplotsFile(yVarName + "_k" + dgDegree + "_2D.tex");

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [14]:
plot.Regression()

index,value
0,"[AgglomerationThreshold0, 1.3169697486750147]"
1,"[AgglomerationThreshold0.1, 1.2987859502263683]"
2,"[AgglomerationThreshold0.2, 1.2953540569809474]"
3,"[AgglomerationThreshold0.3, 1.3112759518834014]"
4,"[AgglomerationThreshold0.4, 1.3173774426199851]"
5,"[AgglomerationThreshold0.5, 1.392411812603103]"


In [15]:
string xVarName = "Grid:hMin";
string yVarName = "L2err_VelocityX";
int dgDegree = 2;

// Filter rows depending on selection
var Tab3 = Tab2.ExtractRows((iRow,RowEntries)=> 
                //Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0 &&
                Convert.ToDouble(RowEntries["DGdegree:Velocity*"]) == dgDegree
            );

// Sort in ascending order of AgglomerationThreshold
System.Data.DataView view = Tab3.DefaultView;
view.Sort = "AgglomerationThreshold ASC"; 
Tab3 = view.ToTable();

Tab3.Print()


    AgglomerationThreshold DGdegree:Velocity* Grid:hMin            Grid:NoOfCells L2err_VelocityX        L2err_Pressure         
0:  0                      2                  0.008333333333333304 57600          1.1058517121663431E-06 1.4559191642240669E-05 
1:  0                      2                  0.016666666666666607 14400          1.2923345429935593E-05 7.451811702821816E-05  
2:  0                      2                  0.033333333333333215 3600           0.00015279344671051534 0.0004531268167517365  
3:  0                      2                  0.06666666666666665  900            0.0017139783424028445  0.0018008173819011944  
4:  0                      2                  0.1333333333333333   225            0.017945227572187014   0.00707456485796849    
5:  0.1                    2                  0.008333333333333304 57600          1.0812418263592682E-06 1.1243550712629745E-05 
6:  0.1                    2                  0.016666666666666607 14400          1.2493233593388

In [16]:
Plot2Ddata plot = new Plot2Ddata();

In [17]:
var plot = Tab3.ToPlot(xVarName, yVarName,  "AgglomerationThreshold");
plot.LogY = true;
plot.LogX = true;
        for (int i = 0; i < plot.dataGroups.Length; i++){
            var dataGroup = plot.dataGroups[i];
            dataGroup.Format.LineColor = allColors[i % allColors.Length];
            dataGroup.Format.PointType = myPointTypes[i % myPointTypes.Length];
        }   
plot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0 
 
 
 AgglomerationThreshold0 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.1 
 
 
 AgglomerationThreshold0.1 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.2 
 
 
 AgglomerationThreshold0.2 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.3 
 
 
 AgglomerationThreshold0.3 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.4 
 
 
 AgglomerationThreshold0.4 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.5 
 
 
 AgglomerationThreshold0.5

In [18]:
plot.SaveToGIF(yVarName + "_k" + dgDegree + "_2D.png");
plot.SavePgfplotsFile(yVarName + "_k" + dgDegree + "_2D.tex");

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [19]:
plot.Regression()

index,value
0,"[AgglomerationThreshold0, 3.5023534797322076]"
1,"[AgglomerationThreshold0.1, 3.4754130694503527]"
2,"[AgglomerationThreshold0.2, 3.4676364016106356]"
3,"[AgglomerationThreshold0.3, 3.4936411336722397]"
4,"[AgglomerationThreshold0.4, 3.4926394801134615]"
5,"[AgglomerationThreshold0.5, 3.5432788714162613]"


In [20]:
string xVarName = "Grid:hMin";
string yVarName = "L2err_VelocityX";
int dgDegree = 3;

// Filter rows depending on selection
var Tab3 = Tab2.ExtractRows((iRow,RowEntries)=> 
                //Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0 &&
                Convert.ToDouble(RowEntries["DGdegree:Velocity*"]) == dgDegree
            );

// Sort in ascending order of AgglomerationThreshold
System.Data.DataView view = Tab3.DefaultView;
view.Sort = "AgglomerationThreshold ASC"; 
Tab3 = view.ToTable();

Tab3.Print()

    AgglomerationThreshold DGdegree:Velocity* Grid:hMin            Grid:NoOfCells L2err_VelocityX        L2err_Pressure         
0:  0                      3                  0.008333333333333304 57600          2.5655506274471643E-05 0.0031391056769672046  
1:  0                      3                  0.016666666666666607 14400          2.0631089600080414E-06 0.00026644281314588096 
2:  0                      3                  0.033333333333333215 3600           1.8258624311553208E-06 2.6058706885237963E-05 
3:  0                      3                  0.06666666666666665  900            3.3407215314432733E-05 0.00026396784258492104 
4:  0                      3                  0.1333333333333333   225            0.0008957590804721692  0.002594756288918595   
5:  0.1                    3                  0.008333333333333304 57600          3.7789262208029935E-09 2.725275750869413E-07  
6:  0.1                    3                  0.016666666666666607 14400          8.0816294436812

In [21]:
Plot2Ddata plot = new Plot2Ddata();

In [22]:
var plot = Tab3.ToPlot(xVarName, yVarName,  "AgglomerationThreshold");
plot.LogY = true;
plot.LogX = true;
        for (int i = 0; i < plot.dataGroups.Length; i++){
            var dataGroup = plot.dataGroups[i];
            dataGroup.Format.LineColor = allColors[i % allColors.Length];
            dataGroup.Format.PointType = myPointTypes[i % myPointTypes.Length];
        }   
plot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -9 
 
 
 
 
 10 -8 
 
 
 
 
 10 -7 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0 
 
 
 AgglomerationThreshold0 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.1 
 
 
 AgglomerationThreshold0.1 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.2 
 
 
 AgglomerationThreshold0.2 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.3 
 
 
 AgglomerationThreshold0.3 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.4 
 
 
 AgglomerationThreshold0.4 
 
 
 
 
 
 
 
 
 
 
 
 AgglomerationThreshold0.5 
 
 
 AgglomerationThreshold0.5

In [23]:
plot.SaveToGIF(yVarName + "_k" + dgDegree + "_2D.png");
plot.SavePgfplotsFile(yVarName + "_k" + dgDegree + "_2D.tex");

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [24]:
plot.Regression()

index,value
0,"[AgglomerationThreshold0, 1.4268808534420538]"
1,"[AgglomerationThreshold0.1, 4.406626262571366]"
2,"[AgglomerationThreshold0.2, 4.379983799193531]"
3,"[AgglomerationThreshold0.3, 4.344717621235352]"
4,"[AgglomerationThreshold0.4, 4.327107089871076]"
5,"[AgglomerationThreshold0.5, 4.343767835284114]"
